# API Yelp

In [10]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv

In [12]:
load_dotenv()
# Récupérer les valeurs des variables d'environnement pour l'authentification
api_key =  "Bearer " + os.getenv("Yelp_key")

endpoint = "https://api.yelp.com/v3/businesses"
path_search = "/search"
path_reviews = "/reviews"

## Les id  et les avis des restaurants

In [38]:
# Requête pour récupérer les id de restaurants
def businesses_request(city):
    outputs = []
    constructed_url = endpoint + path_search
    headers = {'Authorization': api_key}

    for ofs in range(0, 200, 50):

        query = {'categories': 'restaurants', 'location': city, 'offset': ofs, 'limit':50}

        response = requests.get(constructed_url, params=query, headers=headers)

        if response.status_code == 200:
            data = response.json()

            for business in data['businesses']:
                outputs.append(business)
    print('number of businesses', len(outputs))
    return outputs

In [39]:
businesses = businesses_request(city='Paris')
print(businesses[0])

number of businesses 200
{'id': '-0iLH7iQNYtoURciDpJf6w', 'alias': 'le-comptoir-de-la-gastronomie-paris', 'name': 'Le Comptoir de la Gastronomie', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/czh2IEUeT5FnYko4QgUP8A/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/le-comptoir-de-la-gastronomie-paris?adjust_creative=L5NZ4_Fxx4GNgWW1ELo-sQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=L5NZ4_Fxx4GNgWW1ELo-sQ', 'review_count': 1163, 'categories': [{'alias': 'french', 'title': 'French'}], 'rating': 4.5, 'coordinates': {'latitude': 48.8645157999652, 'longitude': 2.34540185646608}, 'transactions': [], 'price': '€€', 'location': {'address1': '34 rue Montmartre', 'address2': '', 'address3': '', 'city': 'Paris', 'zip_code': '75001', 'country': 'FR', 'state': '75', 'display_address': ['34 rue Montmartre', '75001 Paris', 'France']}, 'phone': '+33142333132', 'display_phone': '+33 1 42 33 31 32', 'distance': 370.8275165771047}


In [41]:
# Requête pour récupérer les reviews des restaurants
def reviews_request(businesses):
    outputs = []
    headers = {'Authorization': api_key}
  
    for business in businesses:
        constructed_url = endpoint + f"/{business['id']}" + path_reviews
        response = requests.get(constructed_url, headers=headers)

        if response.status_code == 200:
            data = response.json()

            for review in data['reviews']:
                outputs.append([
                    business['id'], 
                    business['name'], 
                     'Restaurants', 
                     review['id'], 
                     review['rating'], 
                     review['text'].replace('\n', ' ')])
    print('number of reviews', len(outputs))
    return outputs

In [42]:
reviews = reviews_request(businesses)
print(reviews[0])

number of reviews 600
['-0iLH7iQNYtoURciDpJf6w', 'Le Comptoir de la Gastronomie', 'Restaurants', 'pTbFq-jfZRlkP_2ZjQeYNQ', 5, "I fell in love with French onion soup because of Panera. Back then, I only had $20 in cash, and I'd go to the Trumbull Mall with my friends in high school..."]


In [43]:
df = pd.DataFrame(data=reviews, columns=[
    'business_id',
    'business_name',
    'categories',
    'review_id',
    'stars',
    'text'])

df.to_csv('paris_reviews.csv', index=False)

In [44]:
df.head()

,business_id,business_name,categories,review_id,stars,text
0,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Restaurants,pTbFq-jfZRlkP_2ZjQeYNQ,5,I fell in love with French onion soup because ...
1,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Restaurants,zi9OVQcGzeHJ6_T6jRrwkQ,5,"If you like all things duck, then this is your..."
2,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Restaurants,ZwzpceATTrUM7Brv90_usg,5,"If you like foie gras and duck, then you will ..."
3,FFz-WusZrBYZexKqhqzCkg,L'As du Fallafel,Restaurants,bUWPuqCtUDkGV2jwawAlqQ,5,This little gem is our go to spot when visitin...
4,FFz-WusZrBYZexKqhqzCkg,L'As du Fallafel,Restaurants,xdN-zosusXBS0z5ak7O9jg,4,Great spot for a meal on the go that ends up b...


In [45]:
new_df = pd.read_csv('paris_reviews.csv')
new_df.head()

,business_id,business_name,categories,review_id,stars,text
0,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Restaurants,pTbFq-jfZRlkP_2ZjQeYNQ,5,I fell in love with French onion soup because ...
1,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Restaurants,zi9OVQcGzeHJ6_T6jRrwkQ,5,"If you like all things duck, then this is your..."
2,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Restaurants,ZwzpceATTrUM7Brv90_usg,5,"If you like foie gras and duck, then you will ..."
3,FFz-WusZrBYZexKqhqzCkg,L'As du Fallafel,Restaurants,bUWPuqCtUDkGV2jwawAlqQ,5,This little gem is our go to spot when visitin...
4,FFz-WusZrBYZexKqhqzCkg,L'As du Fallafel,Restaurants,xdN-zosusXBS0z5ak7O9jg,4,Great spot for a meal on the go that ends up b...
